<a href="https://colab.research.google.com/github/YomnaAhmed97/HackerRank/blob/master/NOFML%26DS_Final_Practical_Exam_Yomna_Ahmed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b>In this practical exam you are required to do the following:</b><b>
1. Implement the Adam (Batch Version) algorithm to train a multivariable linear regression model.
2. Update your implementation to work for mini-batch.
</b>

### Please fulfil the following requirements:
<br>
<b>
    
    1. Each implementation must be in a sepearate function. i.e. two separate functions.
    2. Check each function on the data in the attached file.
    3. You must obtain r2-score > 0.9 using each functiion.
    4. You must plot the following curves.
        - Loss vs. iterations.
        - Loss vs. each parameter (i.e. loss vs theta 0, loss vs. theta 1 .... etc.).
    5. Choose hyper parameters to obtain the required accuracy.
    6. The implementation must be vectoried and calculations must be performed as matrix vector multiplications.
    7. Apply the stop conditions (gradient < 0.001 and cost convergence check).                                            
</b>
    
<b><i>Note: You are allowed to use the code you developed during practical sessions. However, it should be your own code.</i></b>

## Best of Luck...

# Connect to Drive to upload data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Upload libreries

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
from sklearn.metrics import r2_score

%matplotlib inline

# Upload data

In [4]:
df = pd.read_excel('/content/drive/MyDrive/dataexcel/MultipleLR.csv.xlsx')

In [5]:
df

,73.0,80.0,75.0,152.0
0,93.0,88.0,93.0,185.0
1,89.0,91.0,90.0,180.0
2,96.0,98.0,100.0,196.0
3,73.0,66.0,70.0,142.0
4,53.0,46.0,55.0,101.0
5,69.0,74.0,77.0,149.0
6,47.0,56.0,60.0,115.0
7,87.0,79.0,90.0,175.0
8,79.0,70.0,88.0,164.0
9,69.0,70.0,73.0,141.0


Mini-Batch Gradient descent

In [6]:
def mini_batch(X, Y, mini_batch_size = 100, seed = 0):
   
    
    np.random.seed(seed)            
    m = X.shape[1]                 
    mini_batches = []
        
    permutation = df
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((1,m))

   
    num_complete_minibatches = math.floor(m/mini_batch_size) 
    for k in range(0, num_complete_minibatches):
        
        mini_batch_X = shuffled_X[:, k*mini_batch_size : (k+1)*mini_batch_size]
        mini_batch_Y = shuffled_Y[:, k*mini_batch_size : (k+1)*mini_batch_size]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    if m % mini_batch_size != 0:
  
        mini_batch_X = shuffled_X[:, num_complete_minibatches*mini_batch_size : ]
        mini_batch_Y = shuffled_Y[:, num_complete_minibatches*mini_batch_size : ]
        
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

 Adam

In [7]:
def initialize_adam(parameters) :
 
    
    L = len(parameters) // 2 
    v = {}
    s = {}
    
    
    for l in range(L):
   
        v["dW" + str(l+1)] = np.zeros(parameters['W' + str(l+1)].shape)
        v["db" + str(l+1)] = np.zeros(parameters['b' + str(l+1)].shape)
        s["dW" + str(l+1)] = np.zeros(parameters['W' + str(l+1)].shape)
        s["db" + str(l+1)] = np.zeros(parameters['b' + str(l+1)].shape)
    
    return v, s

In [8]:

def update_parameters_with_adam(parameters, grads, v, s, t, learning_rate = 0.001,
                                beta1 = 0.9, beta2 = 0.999,  epsilon = 1e-8):
 
    
    L = len(parameters) // 2                
    v_corrected = {}                       
    s_corrected = {}                         
    
    for l in range(L):

        v["dW" + str(l+1)] = beta1 * v['dW' + str(l+1)] + (1 - beta1) * grads['dW' + str(l+1)]
        v["db" + str(l+1)] = beta1 * v['db' + str(l+1)] + (1 - beta1) * grads['db' + str(l+1)]

        v_corrected["dW" + str(l+1)] = v["dW" + str(l+1)] / float(1 - beta1**t)
        v_corrected["db" + str(l+1)] = v["db" + str(l+1)] / float(1 - beta1**t)

        s["dW" + str(l+1)] = beta2 * s['dW' + str(l+1)] + (1 - beta2) * (grads['dW' + str(l+1)]**2)
        s["db" + str(l+1)] = beta2 * s['db' + str(l+1)] + (1 - beta2) * (grads['db' + str(l+1)]**2)

        s_corrected["dW" + str(l+1)] = s["dW" + str(l+1)] / float(1 - beta2**t)
        s_corrected["db" + str(l+1)] = s["db" + str(l+1)] / float(1 - beta2**t)



        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * v_corrected["dW" + str(l+1)] / (np.sqrt(s_corrected["dW" + str(l+1)]) + epsilon)
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * v_corrected["db" + str(l+1)] / (np.sqrt(s_corrected["db" + str(l+1)]) + epsilon)
      

    return parameters, v, s

Build model with diffrent optimizers

In [9]:
def model(X, Y, layers_dims, optimizer, learning_rate = 0.001, mini_batch_size = 100, 
          beta1 = 0.9, beta2 = 0.999,  epsilon = 1e-8, num_epochs = 10000, print_cost = True):
 

    L = len(layers_dims)            
    costs = []                       
    t = 0                           
    seed = 10                       
    parameters = (layers_dims)

    if optimizer == "adam":
        v, s = initialize_adam(parameters)
    
    
    for i in range(num_epochs):
        
        seed = seed + 1
        minibatches = mini_batch(X, Y, mini_batch_size, seed)

        for minibatch in minibatches:

            
            (minibatch_X, minibatch_Y) = minibatch

            a3, caches = (minibatch_X, parameters)

            cost = (a3, minibatch_Y)

          
            grads = (minibatch_X, minibatch_Y, caches)

         
            if optimizer == "adam":
                t = t + 1 
                parameters, v, s = update_parameters_with_adam(parameters, grads, v, s,
                                                               t, learning_rate, beta1, beta2,  epsilon)
        if print_cost and i % 1000 == 0:
            print ("Cost after epoch %i: %f" %(i, cost))
        if print_cost and i % 100 == 0:
            costs.append(cost)
                
  
    plt.plot(costs)
    plt.ylabel('cost')
    plt.xlabel('epochs (per 100)')
    plt.title("Learning rate = " + str(learning_rate))
    plt.show()

    return parameters